## Data Cleaning

In [1]:
import pandas as pd
import numpy as np
# read data
rent = pd.read_csv('fang88_rent.csv',header = 0, na_values=["NA","NaN", " ","","NULL","N/A","None"])
house = pd.read_csv('home.csv',header = 0, na_values=["NA","NaN", " ","", "NULL","N/A","None"])

c:\users\hanyuan chi\anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
#pick the columns to use in rent(unique_id and rentzestimate_amount) and check the statistics of rent dataset
rent_use = rent.iloc[:,4:6] #rent_use = rent[["rentzestimate_amount","unique_id"]]
print(rent_use.describe())
print(rent_use.dtypes)
print(rent_use.shape)

       rentzestimate_amount
count         960854.000000
mean            2994.374168
std             4020.805179
min              400.000000
25%             1550.000000
50%             2200.000000
75%             3200.000000
max           250000.000000
rentzestimate_amount    float64
unique_id                object
dtype: object
(1048575, 2)


In [3]:
#remove duplicated unique_id in rent
print(rent_use[rent_use.duplicated(subset = 'unique_id',keep = 'first') == True])
rent_use = rent_use.drop_duplicates(subset = 'unique_id',keep = 'first') #4 duplicates removed

        rentzestimate_amount    unique_id
1902                  1800.0  5.74132E+13
8565                     NaN  5.81157E+13
9169                     NaN  5.82614E+13
422191                2962.0  5.89261E+13


In [4]:
#check the statistics of house dataset and remove duplicated unique_id in house
print(house.describe())
print(house.dtypes)
print(house[house.duplicated(subset = 'unique_id',keep = 'first') == True])#no duplicates found

           bathrooms       bedrooms    list_price       latitude  \
count  349570.000000  338944.000000  3.522200e+05  338114.000000   
mean        2.478044       3.364178  6.903779e+05      33.730169   
std         1.339578       1.313034  1.763565e+06       5.998713   
min         0.000000       0.000000  0.000000e+00     -81.265730   
25%         2.000000       3.000000  2.199990e+05      29.495881   
50%         2.000000       3.000000  3.699000e+05      33.208160   
75%         3.000000       4.000000  6.499000e+05      37.939740   
max        99.990000      99.000000  2.500000e+08      49.002033   

           longitude      lot_sqft          sqft    year_built  
count  338114.000000  2.141130e+05  3.268170e+05  3.360800e+05  
mean      -93.934758  1.301220e+05  2.368388e+03  2.047163e+03  
std        16.612686  8.698167e+06  6.622419e+03  3.375856e+04  
min      -159.676034  0.000000e+00  0.000000e+00  0.000000e+00  
25%       -97.995093  5.600000e+03  1.344000e+03  1.970000e+03

In [5]:
#left join house with rent_use by unique_id
house_rent = house.merge(rent_use,left_on = 'unique_id',right_on = 'unique_id',how = 'left')
print(house_rent.describe())
print(house_rent.dtypes)
print(house_rent.shape)
house_rent.head(5)

           bathrooms       bedrooms    list_price       latitude  \
count  349570.000000  338944.000000  3.522200e+05  338114.000000   
mean        2.478044       3.364178  6.903779e+05      33.730169   
std         1.339578       1.313034  1.763565e+06       5.998713   
min         0.000000       0.000000  0.000000e+00     -81.265730   
25%         2.000000       3.000000  2.199990e+05      29.495881   
50%         2.000000       3.000000  3.699000e+05      33.208160   
75%         3.000000       4.000000  6.499000e+05      37.939740   
max        99.990000      99.000000  2.500000e+08      49.002033   

           longitude      lot_sqft          sqft    year_built  \
count  338114.000000  2.141130e+05  3.268170e+05  3.360800e+05   
mean      -93.934758  1.301220e+05  2.368388e+03  2.047163e+03   
std        16.612686  8.698167e+06  6.622419e+03  3.375856e+04   
min      -159.676034  0.000000e+00  0.000000e+00  0.000000e+00   
25%       -97.995093  5.600000e+03  1.344000e+03  1.97000

,unique_id,bathrooms,bedrooms,city,list_price,latitude,longitude,property_type,lot_sqft,sqft,state,year_built,zip,rentzestimate_amount
0,ABOR_15646314,1.0,1.0,Austin,225000.0,30.388230,-97.965567,RESI,0.0,704.0,TX,1955.0,78734,1350.0
1,ABOR_16633908,5.0,3.0,Burnet,6850000.0,30.674729,-98.212187,RESI,640.0,4889.0,TX,2000.0,78611,9320.0
2,ABOR_17615028,3.0,3.0,San Marcos,334900.0,29.866884,-97.968936,COND,0.0,2351.0,TX,2016.0,78666,NaN
3,ABOR_18247858,2.0,2.0,Austin,735000.0,30.265435,-97.737834,APT,0.0,1416.0,TX,2004.0,78701,3569.0
4,ABOR_18359329,3.0,4.0,Austin,2800000.0,30.325342,-97.779271,RESI,1.0,2016.0,TX,1962.0,78746,13766.0


In [6]:
print(len(house_rent[house_rent.isnull().any(axis=1)]))#check the number of NAs
#fill NA with median for numerical columns expect for zip and rentzestimate_amount
house_rent.iloc[:,:12] = house_rent.iloc[:,:12].fillna(house_rent.iloc[:,:12].median()) 
# #rename the 14th column as rentzestimate_amount
# house_rent.columns.values[13]= 'rentzestimate_amount'
house_rent.columns = house_rent.columns.str.strip() #strip white space from the column names

244862


In [7]:
#Deal with outliers and abnormal values in each column
#bathrooms: capping of outliers--0.9999
q0,q1,q99_99,q100 = np.percentile(house_rent.bathrooms,[0,1,99.99,100])
print(q0,q1,q99_99,q100)
house_rent.loc[house_rent.bathrooms > q99_99,'bathrooms'] = q99_99
np.percentile(house_rent.bathrooms,[0,1,99.99,100])

#bedrooms: capping of outliers--0.9999
q0,q1,q99_99,q100 = np.percentile(house_rent.bedrooms,[0,1,99.99,100])
print(q0,q1,q99_99,q100)
house_rent.loc[house_rent.bedrooms > q99_99,'bedrooms'] = q99_99
np.percentile(house_rent.bedrooms,[0,1,99.99,100])

#list_price: capping of outliers--0.9999
q0,q1,q99_99,q100 = np.percentile(house_rent.list_price,[0,1,99.99,100])
print(q0,q1,q99_99,q100)
house_rent.loc[house_rent.list_price > q99_99,'list_price'] = q99_99
np.percentile(house_rent.list_price,[0,1,99.99,100])

#lot_sqft: capping of outliers--0.9995
q0,q1,q99_95,q100 = np.percentile(house_rent.lot_sqft,[0,1,99.95,100])
print(q0,q1,q99_95,q100)
house_rent.loc[house_rent.lot_sqft > q99_95,'lot_sqft'] = q99_95
np.percentile(house_rent.lot_sqft,[0,1,99.95,100])

#sqft: capping of outliers--0.9995
q0,q1,q99_95,q100 = np.percentile(house_rent.sqft,[0,1,99.95,100])
print(q0,q1,q99_95,q100)
house_rent.loc[house_rent.sqft > q99_95,'sqft'] = q99_95
np.percentile(house_rent.sqft,[0,1,99.95,100])

#for year_built, earlier than 1000 and later than 2017 will be replaced by median
house_rent.loc[(house_rent.year_built < float(1000)) | 
               (house_rent.year_built > float(2017)),'year_built'] = house_rent.year_built.median()

#replace hi with HI in state, Hi with HI, Ha with HA and Unk with UNK
house_rent.loc[house_rent.state == 'hi','state'] = "HI"
house_rent.loc[house_rent.state == 'Hi','state'] = "HI"
house_rent.loc[house_rent.state == 'Ha','state'] = "HA"
house_rent.loc[house_rent.state == 'Unk','state'] = "UNK"

#select main states
main_state = ["AK","AL","AR","AZ","CA","CO","CT","DC","DE","FL","GA","HI","IA","ID",
                                              "IL","IN","KS","KY","LA","MA","MD","ME","MI","MN","MO","MS","MT","NC",
                                              "ND","NE","NH","NJ","NM","NV","NY","OH","OK","OR","PA","RI","SC","SD",
                                              "TN","TX","UT","VA", "VT","WA", "WI", "WV", "WY",
                                              "XX","HI","UNK","HA","BJ"]
house_rent = house_rent.loc[house_rent['state'].isin(main_state)]

#choose rows with zip not being NA and has exactly 5 digits
house_rent = house_rent.loc[house_rent['zip'].isnull()== False]
house_rent.zip = house_rent.zip.astype(str).str.extract('(\d{5})').astype(float)
house_rent = house_rent.loc[house_rent['zip'].isnull()== False]
#np.sort(house_rent.zip.unique())

#chose rows with property_type not being NA and replace Other with OTHER
house_rent = house_rent.loc[house_rent['property_type'].isnull()== False]
house_rent.loc[house_rent.property_type == 'Other','property_type'] = 'OTHER'

0.0 0.0 19.0 99.99
0.0 1.0 20.0 99.0
0.0 1450.0 55000000.0 250000000.0
0.0 0.0 6992861.04 2147483647.0
0.0 0.0 20687.21 3092760.0


c:\users\hanyuan chi\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:52: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


## Data Engineering

In [8]:
#check data structure
print(house_rent.shape)
print(house_rent.count())
print(house_rent.dtypes)

(329503, 14)
unique_id               329503
bathrooms               329503
bedrooms                329503
city                    329238
list_price              329503
latitude                329503
longitude               329503
property_type           329503
lot_sqft                329503
sqft                    329503
state                   329503
year_built              329503
zip                     329503
rentzestimate_amount    176420
dtype: int64
unique_id                object
bathrooms               float64
bedrooms                float64
city                     object
list_price              float64
latitude                float64
longitude               float64
property_type            object
lot_sqft                float64
sqft                    float64
state                    object
year_built              float64
zip                     float64
rentzestimate_amount    float64
dtype: object


In [9]:
#change zip code data type from float to string
house_rent.zip = house_rent.zip.astype(str)
#create new variable: year_from_now
import datetime
house_rent['year_from_now'] = datetime.date.today().year - house_rent['year_built']

In [11]:
#categorical variables: create dummies--property type
#zip,city,state have too many levels, so I don't include them in the prediction
state = pd.get_dummies(house_rent['state']).add_suffix('_state')
property_type = pd.get_dummies(house_rent['property_type']).add_suffix('_type')
house_rent_dummy = pd.concat([house_rent,property_type,state],axis = 1)#combine dummy and the original dataset together

In [12]:
#select useful attributes
property_price_all = house_rent_dummy.loc[:,~house_rent_dummy.columns.isin(['Unnamed: 0','unique_id','city','property_type',
                                                                        'state','year_built','zip'])]


In [14]:
#normalization: (x-mean)/std (dummy variable and y(rentzestimate_amount) don't need normalization)
property_price_norm = property_price_all
property_price_norm = property_price_norm[['bathrooms', 'bedrooms', 'list_price', 'latitude', 'longitude',
       'lot_sqft', 'sqft','year_from_now', 'APT_type',
       'COND_type', 'COOP_type', 'Country Homes/Acreag_type', 'FARM_type',
       'LAND_type', 'MULT_type', 'OTHER_type', 'RENT_type', 'RESI_type',
       'Timeshare_type', 'AK_state', 'AL_state', 'AR_state', 'AZ_state',
       'BJ_state', 'CA_state', 'CO_state', 'DE_state', 'FL_state', 'GA_state',
       'HA_state', 'HI_state', 'IA_state', 'ID_state', 'IL_state', 'IN_state',
       'KS_state', 'KY_state', 'LA_state', 'MA_state', 'MD_state', 'MI_state',
       'MN_state', 'MO_state', 'MT_state', 'NC_state', 'NE_state', 'NH_state',
       'NJ_state', 'NM_state', 'NV_state', 'NY_state', 'OH_state', 'OK_state',
       'OR_state', 'PA_state', 'RI_state', 'SC_state', 'SD_state', 'TN_state',
       'TX_state', 'UNK_state', 'UT_state', 'VA_state', 'WA_state', 'WI_state',
       'WV_state', 'WY_state', 'XX_state','rentzestimate_amount']]#change column order
property_price_norm.iloc[:,:8]=(property_price_norm.iloc[:,:8]- property_price_norm.iloc[:,:8].mean())/(property_price_norm.iloc[:,:8].std())

In [16]:
#property_price_valid -- with rentzestimate_amount value| property_price_tobe -- rentzestimate_amount missing
property_price_valid = property_price_norm.loc[property_price_all['rentzestimate_amount'].isnull()==False]
property_price_tobe = property_price_norm.loc[property_price_all['rentzestimate_amount'].isnull()==True]

#train_set(70% out of property_price_valid), test_set, predict_set
train_set = property_price_valid.sample(frac=0.7,random_state=0)
test_set = property_price_valid.drop(train_set.index)
train_set = train_set.reset_index(drop=True)#reset index of dataframe
test_set = test_set.reset_index(drop=True)
predict_set = property_price_tobe.reset_index(drop=True)
predict_set.iloc[:-1]=predict_set.iloc[:-1].fillna(0)
#fill NA in rentzestimate_amount with 0
predict_set[['rentzestimate_amount']]= predict_set[['rentzestimate_amount']].fillna(0)

print(train_set.shape)
print(test_set.shape)
print(predict_set.shape)

#save train_set, test_set, predict_set to csv
train_set.to_csv('train_prediction.csv',header = False,index = False)
test_set.to_csv('test_prediction.csv',header = False,index = False)
predict_set.to_csv('tobe_prediction.csv',header = False,index = False)

(123494, 69)
(52926, 69)
(153083, 69)


## Neural Network Prediction

In [17]:
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

In [18]:
tf.logging.set_verbosity(tf.logging.INFO) #enable logging--show iteration process
LEARNING_RATE = 0.005

In [19]:
#create the structure of neural network with a function: model_fn
def model_fn(features, targets, mode, params):
    """Model function for Estimator."""

  # Connect the first hidden layer to input layer(features) with relu activation
    first_hidden_layer = tf.contrib.layers.fully_connected(features, 30,tf.nn.relu)

  # Connect the second hidden layer to first hidden layer with relu
    second_hidden_layer = tf.contrib.layers.fully_connected(first_hidden_layer, 30,tf.nn.relu)
    
#     third_hidden_layer = tf.contrib.layers.fully_connected(second_hidden_layer, 30,tf.nn.relu)
    
#     fourth_hidden_layer = tf.contrib.layers.fully_connected(third_hidden_layer, 30,tf.nn.relu)
 
#     fifth_hidden_layer = tf.contrib.layers.fully_connected(fourth_hidden_layer, 30,tf.nn.relu)
  
#     sixth_hidden_layer = tf.contrib.layers.fully_connected(fifth_hidden_layer, 30,tf.nn.relu)
    
#     seventh_hidden_layer = tf.contrib.layers.fully_connected(sixth_hidden_layer, 30,tf.nn.relu)
  
#     eighth_hidden_layer = tf.contrib.layers.fully_connected(seventh_hidden_layer, 30,tf.nn.relu)
   
#     ninth_hidden_layer = tf.contrib.layers.fully_connected(eighth_hidden_layer, 30,tf.nn.relu)
 
#     tenth_hidden_layer = tf.contrib.layers.fully_connected(ninth_hidden_layer, 30,tf.nn.relu)

  # Connect the output layer to second hidden layer (no activation fn)
    output_layer = tf.contrib.layers.linear(second_hidden_layer, 1)

  # Reshape output layer to 1-dim Tensor to return predictions
    predictions = tf.reshape(output_layer, [-1])
    predictions_dict = {"rent": predictions}

  # Calculate loss using MAPE(mean absolute percentage error)
    loss = tf.reduce_mean(tf.abs(1 - (predictions / tf.cast(targets,tf.float32))))
 
  # Calculate mean squared error,root mean squared error, mean absolute error and mape as additional eval metric
    eval_metric_ops = {
      "mse":
          tf.metrics.mean_squared_error(
              tf.cast(targets, tf.float32), predictions),
      "rmse":
          tf.metrics.root_mean_squared_error(
              tf.cast(targets, tf.float32), predictions),
      "mae":
          tf.metrics.mean_absolute_error(
              tf.cast(targets, tf.float32), predictions),
      "mape":
          tf.contrib.keras.metrics.mean_absolute_percentage_error(
          tf.cast(targets, tf.float32), predictions)}
  
  # Optimization algorithm to use during training: Adam
    train_op = tf.contrib.layers.optimize_loss(
      loss=loss,
      global_step=tf.contrib.framework.get_global_step(),
      learning_rate=params["learning_rate"],
      optimizer="Adam")

    return model_fn_lib.ModelFnOps(
      mode=mode, #context in which the model_fn was invoked: fit()/evaluate()/predict()
      predictions=predictions_dict,
      loss=loss,
      train_op=train_op,
      eval_metric_ops=eval_metric_ops)

In [20]:
# Read training_set, testing_set, prediction_set as np.array and define target and features for each dataset
training_set = tf.contrib.learn.datasets.base.load_csv_without_header(
      filename="train_prediction.csv", target_dtype=np.float32, features_dtype=np.float32)

testing_set = tf.contrib.learn.datasets.base.load_csv_without_header(
      filename="test_prediction.csv", target_dtype=np.float32, features_dtype=np.float32)

prediction_set = tf.contrib.learn.datasets.base.load_csv_without_header(
      filename="tobe_prediction.csv", target_dtype=np.float32, features_dtype=np.float32)

# Set model params
model_params = {"learning_rate": LEARNING_RATE}

# Instantiate estimator by calling tf.contrib.learn.Estimator
nn = tf.contrib.learn.Estimator(model_fn=model_fn, params=model_params)

# Use input functions(input_fn) to feed feature (x) and label (y) Tensors into the model for training (get_train_inputs())
def get_train_inputs():
        x = tf.constant(training_set.data)
        y = tf.constant(training_set.target)
        return x, y

# Fit the estimator-nn on training data, no of iterations -- step
nn.fit(input_fn=get_train_inputs, steps=10000)
 
# Use input functions(input_fn) to feed feature (x) and label (y) Tensors into the model for evaluation (get_test_inputs())
def get_test_inputs():
        x = tf.constant(testing_set.data)
        y = tf.constant(testing_set.target)
        return x, y
# Evaluate the estimator-nn on testing data
ev = nn.evaluate(input_fn=get_test_inputs, steps=1)

# Print metrics
print("Loss: %s" % ev["loss"])
print("Mean Squared Error: %s" % ev["mse"])
print("Root Mean Squared Error: %s" % ev["rmse"])
print("Mean Absolute Error: %s" % ev["mae"])
print("Mean Absolute Percentage Error: %s" % ev["mape"])
    
# #print out predictions
# predictions = nn.predict(x=prediction_set.data, as_iterable=True)
# for i, p in enumerate(predictions):
#         print("Prediction %s: %s" % (i + 1, p["rent"]))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_id': 0, '_save_summary_steps': 100, '_num_worker_replicas': 0, '_master': '', '_save_checkpoints_steps': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000025C2CD7F1D0>, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_environment': 'local', '_tf_random_seed': None, '_evaluation_master': '', '_num_ps_replicas': 0, '_model_dir': 'C:\\Users\\HANYUA~1\\AppData\\Local\\Temp\\tmpfi1_jdo_', '_save_checkpoints_secs': 600, '_task_type': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_session_config': None, '_is_chief': True}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\HANYUA~1\AppData\Local\Temp\tmpfi1_jdo_\model.ckpt.
INFO:tensorflow:loss = 0.999977, step = 1
INFO:tensorflow:global_step/sec: 2.5166
INFO:tensorflow:loss = 0.585823, step = 101 (39.736 sec)
INFO:tensorflow:globa

INFO:tensorflow:global_step/sec: 4.69094
INFO:tensorflow:loss = 0.132158, step = 7001 (21.302 sec)
INFO:tensorflow:global_step/sec: 4.29134
INFO:tensorflow:loss = 0.13212, step = 7101 (23.318 sec)
INFO:tensorflow:global_step/sec: 4.71224
INFO:tensorflow:loss = 0.132067, step = 7201 (21.206 sec)
INFO:tensorflow:global_step/sec: 4.14288
INFO:tensorflow:loss = 0.132025, step = 7301 (24.138 sec)
INFO:tensorflow:global_step/sec: 4.67034
INFO:tensorflow:loss = 0.131991, step = 7401 (21.412 sec)
INFO:tensorflow:global_step/sec: 4.5738
INFO:tensorflow:loss = 0.131938, step = 7501 (21.864 sec)
INFO:tensorflow:global_step/sec: 4.0163
INFO:tensorflow:loss = 0.1319, step = 7601 (24.899 sec)
INFO:tensorflow:Saving checkpoints for 7605 into C:\Users\HANYUA~1\AppData\Local\Temp\tmpfi1_jdo_\model.ckpt.
INFO:tensorflow:global_step/sec: 3.9834
INFO:tensorflow:loss = 0.131879, step = 7701 (25.107 sec)
INFO:tensorflow:global_step/sec: 4.28187
INFO:tensorflow:loss = 0.131854, step = 7801 (23.351 sec)
INFO:

In [21]:
# Predict on prediction_set
predictions_predict = nn.predict(x=prediction_set.data, as_iterable=True)
# .predict() returns an iterator; convert to a list
import itertools
predictions_list_predict = list(itertools.islice(predictions_predict,predict_set.shape[0]))
# Combine predictions with prediction_set and save it to csv
tf_predict_with_prediction = pd.concat([predict_set,pd.DataFrame(predictions_list_predict)],axis = 1)
pd.DataFrame(tf_predict_with_prediction).to_csv('tf_predict_with_prediction_with_state.csv',index = False)

# Predictions for training dataset, combine with training set and save it to csv
predictions_train = nn.predict(x=training_set.data, as_iterable=True)
predictions_list_train = list(itertools.islice(predictions_train,train_set.shape[0]))
tf_train_with_prediction = pd.concat([train_set,pd.DataFrame(predictions_list_train)],axis = 1)
pd.DataFrame(tf_train_with_prediction).to_csv('tf_train_with_prediction_with_state.csv',index = False)

# Predictions for testing dataset, combine with testing set and save it to csv
predictions_test = nn.predict(x=testing_set.data, as_iterable=True)
predictions_list_test = list(itertools.islice(predictions_test,test_set.shape[0]))
tf_test_with_prediction = pd.concat([test_set,pd.DataFrame(predictions_list_test)],axis = 1)
pd.DataFrame(tf_test_with_prediction).to_csv('tf_test_with_prediction_with_state.csv',index = False)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Restoring parameters from C:\Users\HANYUA~1\AppData\Local\Temp\tmpfi1_jdo_\model.ckpt-10000
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Restoring parameters from C:\Users\HANYUA~1\AppData\Local\Temp\tmpfi1_jdo_\model.ckpt-10000
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SK

In [22]:
#Take a look of the actual rent and predicted rent for training set, testing set and prediction set
print('train:')
print(tf_train_with_prediction[['rentzestimate_amount','rent']].head())
print('test:')
print(tf_test_with_prediction[['rentzestimate_amount','rent']].head())
print('predict:')
print(tf_predict_with_prediction[['rentzestimate_amount','rent']].head())

train:
   rentzestimate_amount         rent
0                1726.0  1857.768921
1                1875.0  1928.354980
2                1500.0  1462.379639
3                2500.0  2239.605469
4                2500.0  2431.782227
test:
   rentzestimate_amount         rent
0                3569.0  3469.236572
1                1375.0  1101.411987
2                1600.0  1638.502197
3                1600.0  1441.999634
4                1918.0  2122.803955
predict:
   rentzestimate_amount          rent
0                   0.0   2079.402100
1                   0.0   1885.790894
2                   0.0   2322.862061
3                   0.0   1270.009521
4                   0.0  10539.256836
